In [1]:
# run this to shorten the data import from the files
path_data = '/home/nero/Documents/Estudos/DataCamp/Python/courses/Machine_Translation_in_Python/datasets/'

with open(path_data + 'vocab_en.txt', 'r') as en:
    en_text = en.read().split("\n")[:150]
    en.close

with open(path_data + 'vocab_fr.txt', 'r') as fr:
    fr_text = fr.read().split("\n")[:150]
    fr.close

In [35]:
# exercise 01

"""
Defining the Teacher Forcing model layers

You will be defining a new-and-improved version of the machine translation model that you defined earlier. Did you know that models like the Google Machine Translator used this Teacher Forcing technique to train their model?

As you have already seen, your previous model needs to change slightly to adopt Teacher Forcing. In this exercise, you will make the necessary changes to the previous model. You have been provided with the language parameters en_len and fr_len (length of a padded English/French sentences), en_vocab and fr_vocab (Vocabulary size of English/French datasets) and hsize (the hidden layer size of the GRU layers). Remember that the decoder will accept a French sequence with one item less than fr_len. Remember that we use the prefix en to refer to encoder related things and de for decoder related things.
"""

# Instructions

"""

    Import the layers submodule from tensorflow.keras.
    Get the encoder output and state values and assign them to en_out and en_state respectively.
    Define a decoder Input layer which accepts a fr_len-1 long sequence of onehot encoded French words.
    Define a TimeDistributed Dense softmax layer with fr_vocab nodes.

"""
en_len = 20
en_vocab = 200
fr_len = 20
fr_vocab = 250
hsize = 64
# solution

# Import the layers submodule from keras
import tensorflow.keras.layers as layers

en_inputs = layers.Input(shape=(en_len, en_vocab))
en_gru = layers.GRU(hsize, return_state=True)
# Get the encoder output and state
en_out, en_state = en_gru(en_inputs)

# Define the decoder input layer
de_inputs = layers.Input(shape=(fr_len-1, fr_vocab))
de_gru = layers.GRU(hsize, return_sequences=True)
de_out = de_gru(de_inputs, initial_state=en_state)
# Define a TimeDistributed Dense softmax layer with fr_vocab nodes
de_dense = layers.TimeDistributed(layers.Dense(fr_vocab, activation='softmax'))
de_pred = de_dense(de_out)

#----------------------------------#

# Conclusion

"""
Congratulations! You defined all the Keras layers of a machine translation model, that will be trained using Teacher Forcing. Next you will create and compile the model.
"""

2023-08-01 15:41:27.216306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-01 15:41:27.220381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-01 15:41:27.223798: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

'\nCongratulations! You defined all the Keras layers of a machine translation model, that will be trained using Teacher Forcing. Next you will create and compile the model.\n'

In [36]:
# exercise 02

"""
Defining the Teacher Forcing model

With all the layers created, the next step would be to define a Keras Model object. This model is slightly different to the one that you defined earlier, as the new model has two input layers.

You have been provided with the Keras layers that you implemented in the last exercise including en_inputs, en_gru, de_inputs, de_gru and de_pred.
"""

# Instructions

"""

    Import Keras Model object from the models submodule.
    Define a model that takes the encoder input layer and decoder input layer as inputs (in that order) and outputs the final prediction.
    Compile the model using the optimizer adam and the loss function categorical_crossentropy.
    Print the summary of the model.

"""

# solution

# Import the Keras Model object
from tensorflow.keras.models import Model

# Define a model
nmt_tf = Model(inputs=[en_inputs, de_inputs], outputs=de_pred)
# Compile the model with optimizer and loss
nmt_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["acc"])
# Print the summary of the model
nmt_tf.summary()

#----------------------------------#

# Conclusion

"""
Nice work! Keras is great because it allows you to define complex models that accepts multiple inputs and multiple outputs. Next step is to use this model and train it using Teacher Forcing.
"""

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 20, 200)]    0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 19, 250)]    0           []                               
                                                                                                  
 gru_8 (GRU)                    [(None, 64),         51072       ['input_10[0][0]']               
                                 (None, 64)]                                                      
                                                                                                  
 gru_9 (GRU)                    (None, 19, 64)       60672       ['input_11[0][0]',         

'\nNice work! Keras is great because it allows you to define complex models that accepts multiple inputs and multiple outputs. Next step is to use this model and train it using Teacher Forcing.\n'

In [7]:
def sents2seqs(input_type, sentences, onehot=False, pad_type='post', reverse=False):
    assert input_type in ["source", "target"]
    if input_type == 'source':
      tokenizer = en_tok
      pad_length = en_len
      vocab_size = en_vocab
    elif input_type == 'target':
      tokenizer = fr_tok
      pad_length = fr_len
      vocab_size = fr_vocab
    
    encoded_text = tokenizer.texts_to_sequences(sentences)
    preproc_text = pad_sequences(encoded_text, padding=pad_type, truncating='post', maxlen=en_len)
    if reverse:
      preproc_text = preproc_text[:,::-1]
      
    if onehot:
        assert vocab_size is not None, "Cannot do to_categorical without num_classes for safety"
        preproc_text = to_categorical(preproc_text, num_classes=vocab_size)
    return preproc_text

In [16]:
import numpy as np

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Define a Keras Tokenizer
en_tok = Tokenizer()
fr_tok = Tokenizer()

# Fit the tokenizer on some text
en_tok.fit_on_texts(en_text)
fr_tok.fit_on_texts(fr_text)

In [17]:
# exercise 03

"""
Preprocessing data

You now need to process the data for our new model which has two inputs and a single output. The two inputs are, the one-hot encoded English words and one-hot encoded French words excluding the last word.

The output would be the one-hot encoded French words excluding the first word. In other words, in the decoder, each input French word has an output, which is the next word. Here you will learn how to implement that.

You have been provided with the sents2seqs() function, en_text and fr_text.
"""

# Instructions

"""

    Obtain a batch of encoder inputs (from i to i+bsize) using the sents2seqs() function (onehot encoded and reversed).
    Obtain a batch of decoder inputs and outputs (from i to i+bsize) using the sents2seqs() function (onehot encoded).
    Separate the decoder inputs (all French words except the last) from de_xy by slicing on the time dimension.
    Separate the decoder outputs (all French words except the first) from de_xy.

"""

# solution

bsize = 250
for i in range(0, len(en_text), bsize):
  # Get the encoder inputs using the sents2seqs() function
  en_x = sents2seqs('source', en_text[i:i+bsize], onehot=True, reverse=True)
  # Get the decoder inputs/outputs using the sents2seqs() function
  de_xy = sents2seqs('target', fr_text[i:i+bsize], onehot=True)
  # Separate the decoder inputs from de_xy
  de_x = de_xy[:,:-1,:]
  # Separate the decoder outputs from de_xy
  de_y = de_xy[:,1:,:]
  
  print("Data from ", i, " to ", i+bsize)
  print("\tnp.argmax() => en_x[0]: ", np.argmax(en_x[0], axis=-1))
  print("\tnp.argmax() => de_x[0]: ", np.argmax(de_x[0], axis=-1))
  print("\tnp.argmax() => de_y[0]: ", np.argmax(de_y[0], axis=-1))

#----------------------------------#

# Conclusion

"""
Good job! Now you know how to compose training data for the new model. You can see that array slicing is a handy concept to know as it is used heavily for creating data. Let's now move on to learn how you can train the model.
"""

Data from  0  to  250
	np.argmax() => en_x[0]:  [ 0  0  0  0  0  0  0 36  4 35  1  3  5 20  2 34 10  1 33 29]
	np.argmax() => de_x[0]:  [ 37  38   1  11  54  26  12  27   5   3   1 105   2  39   0   0   0   0
   0]
	np.argmax() => de_y[0]:  [ 38   1  11  54  26  12  27   5   3   1 105   2  39   0   0   0   0   0
   0]


"\nGood job! Now you know how to compose training data for the new model. You can see that array slicing is a handy concept to know as it is used heavily for creating data. Let's now move on to learn how you can train the model.\n"

In [23]:
with open(path_data + 'vocab_en.txt', 'r') as en:
    en_text = en.read().split("\n")[:1500]
    en.close

with open(path_data + 'vocab_fr.txt', 'r') as fr:
    fr_text = fr.read().split("\n")[:1500]
    fr.close

In [37]:
# exercise 04

"""
Training the model

Did you know that in 2017, Google translate served more than 500 million users daily?

Here, you will train your first Teacher Forced model. Teacher Forcing is commonly used in sequence to sequence models like your neural machine translator to achieve better performance.

You will be provided with the sents2seqs() function, English sentences en_text and French sentences fr_text.
"""

# Instructions

"""

    Get the decoder input which contains the onehot encoded sequences of French words (except for the last word in every sequence).
    Get the decoder output which contains the onehot encoded sequences of French words (except for the first word in every sequence).
    Train the model on a single batch of data.
    Obtain the evaluation metrics (loss and accuracy) for training data en_x, de_x and de_y.

"""

# solution

n_epochs, bsize = 3, 250
data_size = 1500
for ei in range(n_epochs):
  for i in range(0,data_size,bsize):
    en_x = sents2seqs('source', en_text[i:i+bsize], onehot=True, reverse=True)
    de_xy = sents2seqs('target', fr_text[i:i+bsize], onehot=True)
    # Separate the decoder inputs from de_xy
    de_x = de_xy[:,:-1,:]
    # Separate the decoder outputs from de_xy
    de_y = de_xy[:, 1:, :]
    # Train the model on a single batch of data    
    nmt_tf.train_on_batch([en_x,de_x], de_y)    
    # Obtain the eval metrics for the training data
    res = nmt_tf.evaluate([en_x,de_x], de_y, batch_size=bsize, verbose=0)
    print("{} => Train Loss:{}, Train Acc: {}".format(ei+1,res[0], res[1]*100.0))  

#----------------------------------#

# Conclusion

"""
Congratulations! You successfully trained a model using Teacher Forcing. This model will surely beat the model you implemented earlier. But let's improve the model further by adding a validation step.
"""

2023-08-01 15:41:37.206023: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-01 15:41:37.910217: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-01 15:41:37.915496: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tens

1 => Train Loss:5.507811546325684, Train Acc: 1.3894736766815186


2023-08-01 15:41:49.201319: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


1 => Train Loss:5.496211528778076, Train Acc: 2.652631513774395


2023-08-01 15:41:49.758053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


...

2023-08-01 15:41:57.492364: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


3 => Train Loss:5.27402400970459, Train Acc: 41.85263216495514


2023-08-01 15:41:58.138227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


3 => Train Loss:5.241185665130615, Train Acc: 43.32631528377533


"\nCongratulations! You successfully trained a model using Teacher Forcing. This model will surely beat the model you implemented earlier. But let's improve the model further by adding a validation step.\n"

In [38]:
# exercise 05

"""
Splitting training and validation data

You will be creating training and validation datasets. Keeping a validation dataset and monitoring the performance of model on that set is a good practice to avoid overfitting.

For this exercise you have been provided with en_text (English sentences) and fr_text (French sentences).
"""

# Instructions

"""

    Define a sequence of indices using np.arange(), that starts with 0 and has size of en_text.
    Define train_inds as the first train_size set of indices from the sequence of indices.
    Define tr_en and tf_fr, which contains the sentences found at the indices specified by train_inds in the lists en_text and fr_text.
    Define v_en and v_fr which contains the sentences found at the indices specified by valid_inds in the lists en_text and fr_text.

"""

# solution

train_size, valid_size = 800, 200
# Define a sequence of indices from 0 to size of en_text
inds = np.arange(len(en_text))
np.random.shuffle(inds)
# Define train_inds as first train_size indices
train_inds = inds[:train_size]
valid_inds = inds[train_size:train_size+valid_size]
# Define tr_en (train EN sentences) and tr_fr (train FR sentences)
tr_en = [en_text[ti] for ti in train_inds]
tr_fr = [fr_text[ti] for ti in train_inds]
# Define v_en (valid EN sentences) and v_fr (valid FR sentences)
v_en = [en_text[vi] for vi in valid_inds]
v_fr = [fr_text[vi] for vi in valid_inds]
print('Training (EN):\n', tr_en[:3], '\nTraining (FR):\n', tr_fr[:3])
print('\nValid (EN):\n', v_en[:3], '\nValid (FR):\n', v_fr[:3])

#----------------------------------#

# Conclusion

"""
Great work! You now know how to split the dataset into training and validation data. Here you are using 80% training data and 20% validation data. The 80-20 principle is a common way to split the dataset to train and validation data.
"""

Training (EN):
 ['his favorite fruit is the grapefruit , but my favorite is the mango.', 'paris is never beautiful during summer , and it is chilly in december .', 'california is pleasant during january , but it is usually chilly in april .'] 
Training (FR):
 ['son fruit préféré est le pamplemousse , mais mon préféré est la mangue .', "paris est jamais belle pendant l' été , et il est froid en décembre .", 'californie est agréable en janvier , mais il est généralement froid en avril .']

Valid (EN):
 ['california is usually freezing during july , but it is never warm in january .', 'california is usually relaxing during summer , but it is usually pleasant in autumn .', 'india is sometimes nice during november , but it is never mild in august .'] 
Valid (FR):
 ['la californie est le gel habituellement en juillet , mais il est jamais chaud en janvier .', "california est relaxant habituellement pendant l' été , mais il est généralement agréable à l' automne .", "l' inde est parfois agréab

'\nGreat work! You now know how to split the dataset into training and validation data. Here you are using 80% training data and 20% validation data. The 80-20 principle is a common way to split the dataset to train and validation data.\n'

In [39]:
# exercise 06

"""
Training the model with validation

Here you will train the model using Teacher Forcing and also perform a validation step. You will train the model for multiple epochs and multiple iterations. Then at the end of each epoch, you will run the validation step and obtain the results.

For this, you have been provided with en_text (English sentences), fr_text (French sentences), the sents2seqs() function and nmt_tf (the compiled model). You also have tr_en and tr_fr (training data) and v_en and v_fr (validation data) loaded already.
"""

# Instructions

"""

    Extract decoder inputs (all words except the last) and outputs (all words except the first) from de_xy.
    Train the model on a single batch of data.
    Create decoder inputs and outputs from the validation data similar to how you did for training data.
    Evaluate the model on the validation dataset to obtain the validation loss and the accuracy.

"""

# solution

for ei in range(n_epochs):
  for i in range(0,train_size,bsize):    
    en_x = sents2seqs('source', tr_en[i:i+bsize], onehot=True, reverse=True)
    de_xy = sents2seqs('target', tr_fr[i:i+bsize], onehot=True)
    # Create a single batch of decoder inputs and outputs
    de_x, de_y = de_xy[:,:-1,:], de_xy[:,1:,:]
    # Train the model on a single batch of data
    nmt_tf.train_on_batch([en_x,de_x], de_y)      
  v_en_x = sents2seqs('source', v_en, onehot=True, reverse=True)
  # Create a single batch of validation decoder inputs and outputs
  v_de_xy = sents2seqs('target', v_fr, onehot=True)
  v_de_x, v_de_y = v_de_xy[:,:-1,:], v_de_xy[:,1:,:]
  # Evaluate the trained model on the validation data
  res = nmt_tf.evaluate([v_en_x,v_de_x], v_de_y, batch_size=valid_size, verbose=0)
  print("{} => Loss:{}, Val Acc: {}".format(ei+1,res[0], res[1]*100.0))

#----------------------------------#

# Conclusion

"""
Great work! You now know train the model while monitoring the accuracy on a validation set. But remember to also use a test dataset to measure accuracy on when training models. This is because when you are using the validation dataset to monitor the performance, you are indirectly leaking validation data to the model.
"""

2023-08-02 09:08:30.648590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 09:08:31.518131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 09:08:31.893310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and sh

1 => Loss:5.111137866973877, Val Acc: 42.42105185985565


2023-08-02 09:08:39.158192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 09:08:39.262670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 09:08:39.367531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and sh

2 => Loss:4.853996753692627, Val Acc: 41.947367787361145


2023-08-02 09:08:39.633761: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 09:08:39.746699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 09:08:39.864464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and sh

3 => Loss:4.236155033111572, Val Acc: 41.947367787361145


'\nGreat work! You now know train the model while monitoring the accuracy on a validation set. But remember to also use a test dataset to measure accuracy on when training models. This is because when you are using the validation dataset to monitor the performance, you are indirectly leaking validation data to the model.\n'

In [58]:
# exercise 07

"""
Defining the decoder of the inference model

The inference model is the model that will be used out in the wild to perform translations when required by the user. In this exercise, you will need to implement the decoder of the inference model.

The inference model decoder is different to the decoder of the training model. We can't feed the decoder with French words because that is what we want to predict. Luckily, there is a solution. We can use the predicted French word from the previous time step to feed the inference model decoder. Therefore, when you want to generate a translation, the decoder needs to generate one word at a time, while consuming the previous output as an input.

For this exercise, the variables hsize (hidden size of the GRU layer), fr_len and fr_vocab have been imported. Remember that the prefix de is used to refer to the decoder.
"""

# Instructions

"""

    Define an Input layer which accepts a batch of onehot encoded French word sequences (sequence length 1).
    Define another Input layer which accepts a batch of hsize state, which you will use to feed the previous state to the decoder.
    Get the output and state of the decoder GRU.
    Define a model that accepts the French words Input and the previous state Input and outputs the final prediction and the new GRU state.

"""
hsize = 64
# solution

import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
# Define an input layer that accepts a single onehot encoded word
de_inputs = layers.Input(shape=(1, fr_vocab))
# Define an input to accept the t-1 state
de_state_in = layers.Input(shape=(hsize,))
de_gru = layers.GRU(hsize, return_state=True)
# Get the output and state from the GRU layer
de_out, de_state_out = de_gru(de_inputs, initial_state=de_state_in)
de_dense = layers.Dense(fr_vocab, activation='softmax')
de_pred = de_dense(de_out)

# Define a model
decoder = Model(inputs=[de_inputs, de_state_in], outputs=[de_pred, de_state_out])
print(decoder.summary())

#----------------------------------#

# Conclusion

"""
Great work! Now you have a fully functional encoder-decoder parts of a machine translation model trained with Teacher Forcing and that can be used in the wild. Let's see how you can use this to generate translations.
"""

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 1, 250)]     0           []                               
                                                                                                  
 input_19 (InputLayer)          [(None, 64)]         0           []                               
                                                                                                  
 gru_13 (GRU)                   [(None, 64),         60672       ['input_18[0][0]',               
                                 (None, 64)]                      'input_19[0][0]']               
                                                                                                  
 dense_8 (Dense)                (None, 250)          16250       ['gru_13[0][0]']           

2023-08-02 09:35:08.086963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-02 09:35:08.093941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-02 09:35:08.097624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

"\nGreat work! Now you have a fully functional encoder-decoder parts of a machine translation model trained with Teacher Forcing and that can be used in the wild. Let's see how you can use this to generate translations.\n"

In [59]:
tr_en_gru = nmt_tf.layers[2]
tr_de_gru = nmt_tf.layers[3]
tr_de_dense = nmt_tf.layers[4]

In [60]:
# exercise 08

"""
Link between the trained and inference model

Here you will be transferring the trained weights from the trained model to the inference model. In the encoder decoder model, there are three layers with parameters. They are,

    The encoder GRU layer
    The decoder GRU layer
    The decoder Dense layer

The other layers, such as TimeDistributed do not have any parameters, thus don't require the copying of weights.

For this exercise, you have been provided with the trained encoder GRU layer (tr_en_gru), trained decoder GRU (tr_de_gru) and the trained Dense layer (tr_de_dense). You also have access to all the layers of the inference model (including the encoder) such as the encoder GRU layer (en_gru), decoder GRU (de_gru) and the Dense layer (de_dense).
"""

# Instructions

"""

    Load the weights of the trained encoder GRU layer.
    Set the weights of the encoder GRU layer of the inference model.
    Load the weights for the decoder GRU layer (trained) and set the weights in the inference model.
    Load the weights of the decoder Dense layer (trained) and set the weights in the inference model.

"""

# solution

# Load the weights to the encoder GRU from the trained model
en_gru_w = tr_en_gru.get_weights()
# Set the weights of the encoder GRU of the inference model
en_gru.set_weights(en_gru_w)
# Load and set the weights to the decoder GRU
de_gru.set_weights(tr_de_gru.get_weights())
# Load and set the weights to the decoder Dense
de_dense.set_weights(tr_de_dense.get_weights())

#----------------------------------#

# Conclusion

"""
Great work! You successfully copied the weights from the trained model to the inference model. Always remember to perform this step if you have different training and inference models. This will be a difficult error to track down if you miss this step, because the model will still run but the translations will be incorrect.
"""

'\nGreat work! You successfully copied the weights from the trained model to the inference model. Always remember to perform this step if you have different training and inference models. This will be a difficult error to track down if you miss this step, because the model will still run but the translations will be incorrect.\n'

In [62]:
from keras.utils import to_categorical
def word2onehot(tokenizer, word, vocab_size):
    de_seq = tokenizer.texts_to_sequences([[word]])
    de_onehot = to_categorical(de_seq, num_classes=vocab_size)
    de_onehot = np.expand_dims(de_onehot, axis=1)    
    return de_onehot

In [64]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
def probs2word(probs, tok):
    wid = np.argmax(probs[0,:], axis=-1)
    w = tok.index_word[wid]
    return w

In [68]:
import pandas as pd
data = pd.read_hdf('~/tmp/tmptqied93m/ch4.teacher_forced.h5')

In [67]:
# exercise 09

"""
Generating translations

You will now be generating French translations using an inference model trained using Teacher Forcing.

This model (nmt_tf) has been trained for 50 epochs on 100,000 sentences which achieved around 98% accuracy on a 35000+ validation set. It might take longer for this exercise to initialize as the trained model needs to be loaded. You are provided with the sents2seqs() function. You have also been given two new functions:

word2onehot(tokenizer, word, vocab_size) which accepts:

    tokenizer - A Keras Tokenizer object
    word - A string representing a word from the vocabulary (e.g. 'apple')
    vocab_size - Vocabulary size

probs2word(probs, tok) which accepts:

    probs - An output from the model of the shape [1,<French Vocab Size>]
    tok - A Keras Tokenizer object

You can peak at the source code for these functions by typing print(inspect.getsource(word2onehot)) and print(inspect.getsource(probs2word)) in the console.
"""

# Instructions

"""

    Predict the initial decoder state (de_s_t) with the encoder.
    Predict the output and the new state from the decoder using the previous prediction (output) and the previous state as inputs. Remember to recursively generate the new state.
    Get the word string from the probability output using the probs2word() function.
    Convert the word string to a one-hot sequence using the word2onehot() function.

"""

# solution

en_sent = ['the united states is sometimes chilly during december , but it is sometimes freezing in june .']
print('English: {}'.format(en_sent))
en_seq = sents2seqs('source', en_sent, onehot=True, reverse=True)
# Predict the initial decoder state with the encoder
de_s_t = en.predict(en_seq)
de_seq = word2onehot(fr_tok, 'sos', fr_vocab)
fr_sent = ''
for i in range(fr_len):    
  # Predict from the decoder and recursively assign the new state to de_s_t
  de_prob, de_s_t = decoder.predict([de_seq,de_s_t])
  # Get the word from the probability output using probs2word
  de_w = probs2word(de_prob, fr_tok)
  # Convert the word to a onehot sequence using word2onehot
  de_seq = word2onehot(fr_tok, de_w, fr_vocab)
  if de_w == 'eos': break
  fr_sent += de_w + ' '
print("French (Ours): {}".format(fr_sent))
print("French (Google Translate): les etats-unis sont parfois froids en décembre, mais parfois gelés en juin")

#----------------------------------#

# Conclusion

"""
Congratulations! You now know a lot about Teacher Forcing, including how to train a model with Teacher Forcing and how to generate translations from such a model. Teacher Forcing is a commonly used technique for training sequence-to-sequence models.
"""

English: ['the united states is sometimes chilly during december , but it is sometimes freezing in june .']


AttributeError: 'GRU' object has no attribute 'predict'

In [9]:
# exercise 10

"""
Measuring word vector similarity

In this lesson we will understand the power of word vectors using real world trained word vectors. These are word vectors extracted from a list of word vectors published by the Stanford NLP group. A word vector is a sequence or a vector of numerical values. For example, dog = (0.31, 0.92, 0.13)
(https://nlp.stanford.edu/projects/glove/)
The distance between word vectors can be measured using a pair-wise similarity metric. Here we will be using sklearn.metrics.pairwise.cosine_similarity. Cosine similarity produces a higher values when the element-wise similarity of two vectors is high and vice-versa.
"""

# Instructions

"""

    Print the length of the cat_vector using ndarray.size attribute.
    Compute and print the similarity between the cat_vector and window_vector using cosine_similarity.
    Compute and print the similarity between the cat_vector and dog_vector using cosine_similarity.

"""

# solution

from sklearn.metrics.pairwise import cosine_similarity

# Print the length of the cat_vector
print('Length of the cat_vector: ', cat_vector.size)

# Compute and print the similarity between cat and window vectors
dist_cat_window = cosine_similarity(cat_vector, window_vector)
print('Similarity(cat, window): ', dist_cat_window)

# Compute and print the similarity between cat and dog vectors
print('Similarity(cat,dog): ', cosine_similarity(cat_vector, dog_vector))

#----------------------------------#

# Conclusion

"""
Congratulations, you have now learned how to measure the similarity between word vectors. This is quite useful to measure similarity between high dimensional word vectors.
"""

'\n\n'

In [71]:
# exercise 11

"""
Defining the embedding model

You will be defining a Keras model that:

    Uses Embedding layers
    Will be trained with Teacher Forcing

This model will have two embedding layers; an encoder embedding layer and a decoder embedding layer. Furthermore, as the model is trained using Teacher Forcing, it will use a sequence length of fr_len-1 in the decoder Input layer.

For this exercise, you have all the required keras.layers and Model imported. Furthermore the variables, en_len (English sequence length), fr_len (French sequence length), en_vocab (English vocabulary size), fr_vocab (French vocabulary size) and hsize (hidden size) have been defined.
"""

# Instructions

"""

    Define an Input layer which accepts a sequence of word IDs.
    Define an Embedding layer that embeds en_vocab words, has length 96 and can accept a sequence of IDs (sequence length is specified using the input_length argument).
    Define an Embedding layer that embeds fr_vocab words, has length 96 and can accept a sequence of fr_len-1 IDs.
    Define a model that takes an input from the encoder and an input from the decoder (in that order) and outputs the word predictions.

"""

from keras.layers import Input, Embedding, GRU, TimeDistributed, Dense
from keras.models import Model

# solution

# Define an input layer which accepts a sequence of word IDs
en_inputs = Input(shape=(en_len,))
# Define an Embedding layer which accepts en_inputs
en_emb = Embedding(en_vocab, 96, input_length=en_len)(en_inputs)
en_out, en_state = GRU(hsize, return_state=True)(en_emb)

de_inputs = Input(shape=(fr_len-1,))
# Define an Embedding layer which accepts de_inputs
de_emb = Embedding(fr_vocab, 96, input_length=fr_len-1)(de_inputs)
de_out, _ = GRU(hsize, return_sequences=True, return_state=True)(de_emb, initial_state=en_state)
de_pred = TimeDistributed(Dense(fr_vocab, activation='softmax'))(de_out)

# Define the Model which accepts encoder/decoder inputs and outputs predictions 
nmt_emb = Model([en_inputs, de_inputs], de_pred)
nmt_emb.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

#----------------------------------#

# Conclusion

"""
Congratulations! You just implemented a machine translation model that uses an embedding layer. Embeddings are much more powerful than onehot vectors as the capture semantic relationships between words. Therefore you can expect better performance with an embeddings layer.
"""

2023-08-02 10:19:10.415119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-02 10:19:10.420744: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-02 10:19:10.424760: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

'\nCongratulations! You just implemented a machine translation model that uses an embedding layer. Embeddings are much more powerful than onehot vectors as the capture semantic relationships between words. Therefore you can expect better performance with an embeddings layer.\n'

In [72]:
# exercise 12

"""
Training the word embedding based model

Here you will learn how to implement the training process for a machine translator model that uses word embeddings. A word is represented as a single number instead of a one-hot encoded vector as you did in previous exercises. You will train the model for multiple epochs while traversing through the full dataset in batches.

For this exercise you are provided with training data (tr_en and tr_fr) in the form of a list of sentences. You will only use a very small sample (1000 sentences) of the actual data as it can otherwise take very long to train. You also have the sents2seqs() function and the model, nmt_emb, that you implemented in the previous exercise. Remember that we use en_x to refer to encoder inputs and de_x to refer to decoder inputs.


"""

# Instructions

"""

    Get a single batch of French sentences without onehot encoding using the sents2seqs() function.
    Get all the words except the last from de_xy.
    Get all the words except the first from de_xy_oh (French words with onehot encoding).
    Train the model using a single batch of data

"""

# solution

for ei in range(3):
  for i in range(0, train_size, bsize):    
    en_x = sents2seqs('source', tr_en[i:i+bsize], onehot=False, reverse=True)
    # Get a single batch of French sentences with no onehot encoding
    de_xy = sents2seqs('target', tr_fr[i:i+bsize], onehot=False)
    # Get all words except the last word in that batch
    de_x = de_xy[:,:-1]
    de_xy_oh = sents2seqs('target', tr_fr[i:i+bsize], onehot=True)
    # Get all words except the first from de_xy_oh
    de_y = de_xy_oh[:,1:,:]
    # Training the model on a single batch of data
    nmt_emb.train_on_batch([en_x,de_x], de_y)    
    res = nmt_emb.evaluate([en_x, de_x], de_y, batch_size=bsize, verbose=0)
    print("{} => Loss:{}, Train Acc: {}".format(ei+1,res[0], res[1]*100.0))

#----------------------------------#

# Conclusion

"""
Congratulations! You now understand how to use word embedding to improve a machine translation model. You can get reasonably good results (i.e. ~98% validation accuracy) by using the full dataset to train the model. There are also more techniques like making models deeper and introducing an attention mechanism that can greatly improve the performance of machine translation models.
"""

2023-08-02 10:19:25.428556: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]
2023-08-02 10:19:26.154149: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-02 10:19:26.162324: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tens

1 => Loss:5.508350849151611, Train Acc: 3.83157879114151


2023-08-02 10:19:39.094151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


1 => Loss:5.496906757354736, Train Acc: 37.41052746772766


2023-08-02 10:19:39.859046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


...

2023-08-02 10:19:54.219681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [250,19,250]
	 [[{{node Placeholder/_2}}]]


3 => Loss:5.341002941131592, Train Acc: 41.87368452548981


2023-08-02 10:19:54.633589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [50,19,250]
	 [[{{node Placeholder/_2}}]]


3 => Loss:5.296236515045166, Train Acc: 45.36842107772827


'\nCongratulations! You now understand how to use word embedding to improve a machine translation model. You can get reasonably good results (i.e. ~98% validation accuracy) by using the full dataset to train the model. There are also more techniques like making models deeper and introducing an attention mechanism that can greatly improve the performance of machine translation models.\n'